In [44]:
import pandas as pd
prev=pd.read_csv('20170101-20200619_original_final.csv')

In [45]:
prev.tail()

,timestamp,target_value,rainfall,temperature,abnormal,holiday,before_holiday,after_holiday,dayofweek_median,day_sin,menu,menu_nouns,menu_origin,menu_nouns2,scaled_target,weekday,same_menu
1261,2020-06-15,412.0,0.0,28.6,1,0,0,0,353.5,0.781831,제육고추장불고기\n(돼지고기:국내산),제육고추장불고기,돼지고기,불고기,2.000000,0.0,1.862320
1262,2020-06-16,298.0,0.0,30.7,1,0,0,0,330.5,0.974928,전주비빔밥*계란후라이,전주비빔밥 계란후라이,NaN,비빔밥,1.025641,1.0,1.633454
1263,2020-06-17,295.0,0.0,30.3,1,0,0,0,311.0,0.433884,"함박스테이크\n(돼지고기:국내산,닭고기:국내산)",함박스테이크,돼지고기 닭고기,스테이크,1.000000,2.0,1.411616
1264,2020-06-18,NaN,0.0,27.3,0,0,0,0,326.0,-0.433884,단호박영양밥,NaN,NaN,NaN,NaN,3.0,NaN
1265,2020-06-19,NaN,0.1,26.0,0,0,0,0,192.0,-0.974928,물냉면,NaN,NaN,NaN,NaN,4.0,NaN


In [46]:
import pandas as pd
from datetime import datetime
#target=점장님이 입력하는 금주 월-금 식수 데이터 
target=pd.DataFrame({'timestamp':['2020-06-22','2020-06-23','2020-06-24','2020-06-25','2020-06-26'],'target_value':[100,200,300,400,500]})

In [9]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta

url='http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?solYear={year}&solMonth={date}&ServiceKey=J%2FYE3sN528TkttbFXsmjXDfVrpZyKNjduV1r%2FcUxAfJ7wHnrjO2F5zzkJx2JvBHsmuNstAVe%2B5n6fS2dNgcKvQ%3D%3D'.format(year='2020',date='10')
resp=requests.get(url)
soup=BeautifulSoup(resp.content)
Text=soup.find_all('locdate')
Text[0].text
           

'20201001'

In [47]:
#점장님이 입력한 금주 금요일까지 채워지는 함수입니다
def make_df(prev,target):
    
    import numpy as np
    import requests
    from bs4 import BeautifulSoup
    from datetime import datetime
    from datetime import timedelta
    prev['timestamp']=pd.to_datetime(prev['timestamp'])
    n=(datetime.strptime(target.loc[int(target.index[-1]),'timestamp'],"%Y-%m-%d")-prev.loc[int(prev.index[-1]),'timestamp']).days+1
    if n<=0:  #n을 3개로 나눠야 겠다 1.06/19이전 2.06/19~today() 3.today()~미래
        pass
    else:
        num=[]
        for i in range(n):
            num.append(int(prev.index[-1]+i))

    #     num=[i for i in prev[prev['temperature'].isnull()].index]
    #     n=len(num)
        for i in range(n):    
    #         prev.loc[num[i]]=np.nan
            prev.loc[num[i],'timestamp']=prev.loc[num[i]-1,'timestamp']+timedelta(days=1)
            prev.loc[num[i],'day_sin']=prev.loc[num[i]-7,'day_sin']
            prev.loc[num[i],'weekday']=prev.loc[num[i]-7,'weekday']
            prev.loc[num[i],'dayofweek_median']=prev.loc[num[i]-7,'dayofweek_median']

        url='http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?solYear={year}&solMonth={date}&ServiceKey=J%2FYE3sN528TkttbFXsmjXDfVrpZyKNjduV1r%2FcUxAfJ7wHnrjO2F5zzkJx2JvBHsmuNstAVe%2B5n6fS2dNgcKvQ%3D%3D'.format(year=str((prev.loc[prev.index[-1],'timestamp']).year),date=str(prev.loc[prev.index[-1],'timestamp'].month))
        resp=requests.get(url)
        soup=BeautifulSoup(resp.content)
        Text=soup.find_all('locdate')
        holiday=[]
        for i in range(len(Text)):
            holiday.append(pd.to_datetime(Text[i].text))  
        def holiday1(prev):
            if prev['timestamp'] in holiday:
                return 1
            else:
                return 0
        def before_holiday(prev):
            if prev['holiday']==1:
                return 0
            elif prev['timestamp']+timedelta(days=1) in holiday:
                return 1
            else:
                return 0    

        def after_holiday(prev):
            if prev['holiday']==1:
                return 0
            elif prev['timestamp']+timedelta(days=-1) in holiday:
                return 1
            else:
                return 0 

        prev['holiday']=prev.apply(holiday1,axis=1)
        prev['before_holiday']=prev.apply(before_holiday,axis=1)
        prev['after_holiday']=prev.apply(after_holiday,axis=1)   
        
        
        nn=(datetime.strptime(target.loc[int(target.index[-1]),'timestamp'],"%Y-%m-%d")-datetime.strptime(datetime.today().strftime("%Y-%m-%d"),"%Y-%m-%d")).days
    # 미래 temperature         
        from bs4 import BeautifulSoup
        import requests
        bam=requests.get('https://m.weather.naver.com/')
        soup=BeautifulSoup(bam.text,'html.parser')
        content=soup.find_all('div',{'class':'weekly_item_temperature'})
        temp=[]
        for i in range(nn):
            temp.append(content[i].find_all('span')[3].text[6:8]) # 06/18~06/27      

        for i in range(nn):
            prev.loc[prev['timestamp']==datetime.strptime(datetime.today().strftime("%Y-%m-%d"),"%Y-%m-%d")+timedelta(days=i+1),'temperature']=temp[i] 

    # 미래 rainfall
        for i in range(nn):
            bam=requests.get('https://darksky.net/details/37.5396,127.1197/{date}/si12/en'.format(date=datetime.strptime(datetime.today().strftime("%Y-%m-%d"),"%Y-%m-%d")+timedelta(days=i+1)))
            soup=BeautifulSoup(bam.text,'html.parser')
            content=soup.find_all('span',{'class':'num swip'})
            prev.loc[prev['timestamp']==datetime.strptime(datetime.today().strftime("%Y-%m-%d"),"%Y-%m-%d")+timedelta(days=i+1),'rainfall']=content[0].text
        
        # 과거 rainfall
        bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=108&yy=2020&obs=21&x=18&y=7')  
        soup=BeautifulSoup(bam.text,'html.parser')
        content=soup.find_all('td')
        month=[i.month for i in prev[prev['rainfall'].isnull()]['timestamp']]
        day=[i.day for i in prev[prev['rainfall'].isnull()]['timestamp']]
        prev.loc[prev['rainfall'].isnull(),'rainfall']=[i.replace("\xa0", "0") if i =='\xa0' else i for i in [content[3+(day[u]-1)*13+month[u]].text for u in range(len(month))]]   

        # 과거 temperature    
        bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=108&yy=2020&obs=08&x=25&y=4')  
        soup=BeautifulSoup(bam.text,'html.parser')
        content=soup.find_all('td')
        month=[i.month for i in prev[prev['temperature'].isnull()]['timestamp']]
        day=[i.day for i in prev[prev['temperature'].isnull()]['timestamp']]
        prev.loc[prev['temperature'].isnull(),'temperature']=[i for i in [content[3+(day[u]-1)*13+month[u]].text for u in range(len(month))]]


    # target 주말 0으로 채우기     
        def weekend(prev):
            if prev['weekday'] in [5,6]:
                return 0
            else:
                return prev['target_value']
        prev['target_value']=prev.apply(weekend,axis=1) 


    # 점장님이 입력한 금주 월-목 target 채우기   
        for i in range(len(target)):
            prev.loc[prev['timestamp']==datetime.strptime(target['timestamp'][i],"%Y-%m-%d"),'target_value']=target['target_value'][i]
        prev=prev.astype({'temperature' : 'float'})
        prev=prev.astype({'rainfall' : 'float'})   
        prev['temperature']=prev['temperature'].fillna(0)
#         
    # 기온 이상치     
        def abnormal(prev):
            if (prev['timestamp'].month in [6,7,8]) & (float(prev['temperature'])>=28):
                return 1
            elif (prev['timestamp'].month in [9,10,11]) & (float(prev['temperature'])>=5):
                return 1
            elif (prev['timestamp'].month in [12,1,2]) & (float(prev['temperature'])<=-5):
                return 1
            elif (prev['timestamp'].month in [3,4,5]) & (float(prev['temperature'])<=5):
                return 1
            else:
                return 0

        prev['abnormal']=prev.apply(abnormal,axis=1) 

#     스케일링    

        def z_score_normalize(lst):
            normalized_lst = []
            mean = sum(lst)/len(lst)
            std = np.std(lst)
            normalized_lst = [ (value - mean) / std for value in lst]
            return normalized_lst


        def Scaling(DF):
            # 스케일링할 컬럼 이름
            column_lst = ['rainfall', 'temperature','dayofweek_median','weekday','day_sin']

            for column_name in column_lst:
                DF[column_name] = z_score_normalize(DF[column_name])
            return DF

        Scaling(prev)
    return prev

In [48]:
import time
start = time.time()
make_df(prev,target).tail(20)
# print("time :", time.time() - start)

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


,timestamp,target_value,rainfall,temperature,abnormal,holiday,before_holiday,after_holiday,dayofweek_median,day_sin,menu,menu_nouns,menu_origin,menu_nouns2,scaled_target,weekday,same_menu
1253,2020-06-07,0.0,-0.254924,1.370217,0,0,0,0,-1.493269,-0.000869,NaN,NaN,NaN,NaN,0.000000,1.500786,0.000000
1254,2020-06-08,373.0,-0.254924,1.758550,1,0,0,0,0.947044,1.104905,"짬뽕밥\n(돼지고기:국내산,오징어:페루산)",짬뽕밥,돼지고기 오징어,짬뽕밥,2.000000,-1.499215,2.000000
1255,2020-06-09,297.0,-0.254924,1.855633,1,0,0,0,0.788269,1.378009,"오삼불고기\n(돼지고기:국내산,오징어:페루산)",오삼불고기,돼지고기 오징어,오삼불고기,1.796247,-0.999215,1.628823
1256,2020-06-10,321.0,0.287559,1.865341,1,0,0,0,0.653654,0.612789,오므라이스,오므라이스,NaN,오므라이스,1.860590,-0.499215,1.910742
1257,2020-06-11,308.0,-0.254924,1.816800,1,0,0,0,0.757204,-0.614527,돼지갈비찜\n(돼지고기:미국산/국내산),돼지갈비찜,돼지고기,돼지갈비,1.825737,0.000786,1.880759
1258,2020-06-12,159.0,-0.254924,1.428467,0,0,0,0,-0.167837,-1.379746,냉모밀,냉모밀,NaN,냉모밀,1.426273,0.500786,1.426273
1259,2020-06-13,0.0,-0.254924,2.001258,1,0,0,0,-1.493269,-1.106642,NaN,NaN,NaN,NaN,0.000000,1.000786,0.000000
1260,2020-06-14,0.0,-0.158677,1.544966,1,0,0,0,-1.493269,-0.000869,NaN,NaN,NaN,NaN,0.000000,1.500786,0.000000
1261,2020-06-15,412.0,-0.254924,1.525550,1,0,0,0,0.947044,1.104905,제육고추장불고기\n(돼지고기:국내산),제육고추장불고기,돼지고기,불고기,2.000000,-1.499215,1.862320
1262,2020-06-16,298.0,-0.254924,1.729425,1,0,0,0,0.788269,1.378009,전주비빔밥*계란후라이,전주비빔밥 계란후라이,NaN,비빔밥,1.025641,-0.999215,1.633454


In [49]:
print("time :", time.time() - start)

time : 8.420486450195312


In [67]:
    # 미래 temperature  
nn=7
from bs4 import BeautifulSoup
import requests
bam=requests.get('https://n.weather.naver.com/')
soup=BeautifulSoup(bam.text,'html.parser')
content=soup.find_all('strong',{'class':'temperature'})
temp=[]
for i in range(nn):
     temp.append(content[i].text[12:14]) # 06/18~06/27      

# for i in range(nn):
#     prev.loc[prev['timestamp']==datetime.strptime(datetime.today().strftime("%Y-%m-%d"),"%Y-%m-%d")+timedelta(days=i+1),'temperature']=temp[i] 


In [69]:
temp[1:]

['29', '28', '29', '29', '30', '29']

In [77]:
temp=[]
for i in range(1,2):
     temp.append(content[i].text[12:14])

In [78]:
temp

['29']